In [1]:
#java -mx4g -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLPServer -port 9000 -timeout 15000
from pycorenlp import StanfordCoreNLP
import pandas as pd
import re
nlp_wrapper = StanfordCoreNLP('http://localhost:9000')

In [2]:
#open file .html
with open("bankstatement.html", "r", encoding='utf-8') as f:
    text= f.read()
#cleaning
import re
def bersihsub(expr):

    cleanr = re.compile('<.*?>')
    expr = str(expr)
    hsl = re.sub(cleanr,"!",expr)
    hsl = re.sub("!!!!",r"\n",hsl)
    hsl = re.sub("!","",hsl)
#     hsl = re.sub("  ","\n",hsl)
    return hsl

text2 = bersihsub(text)
print(text2)
text_split = text2.split("\n")


DBS Bank Ltd
12 Marina Boulevard, Marina Bay Financial Centre Tower 3, Singapore 018982
www.dbs.com.sg | www.posb.com.sg
ODBS $POSB
S/N: EN05310800573439
CONSOLIDATED STATEMENT
JOHN RIEGER
KTGIF SINGAPORE PTE. LTD.
26B TEMPLE STREET
#03-00
SINGAPORE 058571
For enquiries, please call our 24-Hour Hotline
at 1800-111 1111 (in Singapore) or (65) 632
2265 (outside Singapore)
ACCOUNT SUMMARY | As at 31 Aug 2018
DEPOSITS
MULTI CURRENCY
Balance
Account
Account Number
(S$ Equivalent)
DBS Multiplier Account
12-145753-2
SGD 2,943.76
2,943.76

TOTAL DEPOSITS - CREDIT
2,943.76


DBS Bank Ltd
12 Marina Boulevard, Marina Bay Financial Centre Tower 3, Singapore 018982
www.dbs.com.sg | www.posb.com.sg
8DBS &lt;fPOSB
ACCOUNT DETAILS | As at 31 Aug 2018
DEPOSITS
DBS Multiplier Account
Date
Description
Withdrawal
Deposit
CURRENCY: SINGAPORE DOLLAR
Balance Brought Forward
1.254.12
28 Aug
Quick Cheque Deposit
30 Aug
Point-of-Sale Transaction
20.00
TRANSIT LINK PTE LTD
30 Aug
Point-of-Sale Transaction
465.0

In [3]:
def get_stanfordnlp(sentence):
    annot_doc = nlp_wrapper.annotate(sentence,
    properties={
        'annotators': 'ner, pos',
        'outputFormat': 'json',
        'timeout': 1000,
    })
    return annot_doc

In [4]:
name = []
suspect_date2 = []
suspect_account_number = []
address_suspect=[]
get_transaction=[]
suspect_account_number=[]

loc = ['COUNTRY','LOCATION','CITY']

for i in range (len(text_split)):
    annot_doc = get_stanfordnlp(text_split[i])
#     print(i)
    for sentence in annot_doc["sentences"]:
        m = len(sentence['entitymentions'])
        if m!=0:
            for j in range(m):
#                 print(sentence['entitymentions'][j]['ner'])
                if sentence['entitymentions'][j]['ner'] == "PERSON":
                    suspect_name = []
#                     print(sentence['tokens'])
                    word = text_split[i].split()
                    for w in word:
                        annot_tok = get_stanfordnlp(w)
                        for token in annot_tok['sentences']:
                            if (token['tokens'][0]['ner']) == "PERSON":
                                suspect_name.append(w)
                    name.append(" ".join(suspect_name))
                #get date
                if sentence['entitymentions'][j]['ner'] ==  "DATE":
                    n = len(sentence['tokens'])
                    suspect_date=[]
                    for k in range (n):
                        if sentence['tokens'][k]['ner'] == "DATE":
                            suspect_date.append(sentence['tokens'][k]['word'])
                    if len(suspect_date)>1:
                        suspect_date2.append(" ".join(suspect_date))
                #get number for account
                if sentence['entitymentions'][j]['ner'] ==  "NUMBER":
                    n = len (sentence['tokens'])
                    
                    collection_list=[]
#                     print(n)
                    for k in range(n):
                        collection_list.append(sentence['tokens'][k]['ner'])
                    if "LOCATION" not in collection_list and "COUNTRY" not in collection_list:
                        suspect_account_number.append(text_split[i])
    #because many statement bank paper, the address after name.        
    for n in name:
        if n in text_split[i]:
#             print(text_split[i])
            j=i+1
            suspect_address=[]
            while(j<len(text_split)):
                temp=False
                annot_doc = get_stanfordnlp(text_split[j])
                suspect_address.append(text_split[j])
#                 print(text_split[j])
                for sentence in annot_doc["sentences"]:
            #         for word in sentence["tokens"]:
            #         print(sentence)
            #         print("***")
                    m = len(sentence['entitymentions'])
                    if m!=0:
                        for k in range(m):
#                             print(sentence['entitymentions'][k]['ner'])
                            if sentence['entitymentions'][k]['ner'] == "COUNTRY":
                                temp = True
                if temp == True:
                    break
                j+=1
            address_suspect.append(" ".join(suspect_address))
#     info=[]
        
#                     suspect_account_number.append(i)
#     print("_--------_")
#         print (sentence["word"] + "=>" + sentence["ner"])


In [5]:
#get transaction bank
get_transaction=[]
for i in range(len(text_split)):
    if text_split[i] in suspect_date2:
        print(text_split[i])
        j=i+1
        info=[]
        info.append(text_split[i])
        
        while(j<len(text_split)):
#             temp=False
            if text_split[j] not in suspect_date2:
                if text_split[j].lower() in 'total':
                    break
                else:
                    info.append(text_split[j])
            else:
                break
            j+=1
        get_transaction.append(info)
get_transaction

28 Aug
30 Aug
30 Aug
31 Aug
31 Aug


[['28 Aug', 'Quick Cheque Deposit'],
 ['30 Aug', 'Point-of-Sale Transaction', '20.00', 'TRANSIT LINK PTE LTD'],
 ['30 Aug',
  'Point-of-Sale Transaction',
  '465.00',
  'S &amp; S LINKERS PTE LTD'],
 ['31 Aug', 'Point-of-Sale Transaction', '26.50', 'GAYATRI RESTAURANT'],
 ['31 Aug',
  'Point-of-Sale Transaction',
  '16.00',
  'FOCUS NETWORK AGENCIES (S) PTE LTD']]

In [6]:
date=[]
description=[]
amount=[]
type_=[]
for i in get_transaction:
    get_date=[]
    get_desc=[]
    get_amount=[]
    get_info=[]
    for j in i:
        print(j)
        annot_doc = get_stanfordnlp(j)
        for ls in annot_doc['sentences']:
            if ls["tokens"][0]['ner'] == "DATE":
                get_date.append(j)
            if ls["tokens"][0]['ner'] == "NUMBER":
                get_amount.append(j)
            else:
                get_desc.append(j)
    date.append(get_date)
    description.append(get_desc)
    amount.append(get_amount)


28 Aug
Quick Cheque Deposit
30 Aug
Point-of-Sale Transaction
20.00
TRANSIT LINK PTE LTD
30 Aug
Point-of-Sale Transaction
465.00
S &amp; S LINKERS PTE LTD
31 Aug
Point-of-Sale Transaction
26.50
GAYATRI RESTAURANT
31 Aug
Point-of-Sale Transaction
16.00
FOCUS NETWORK AGENCIES (S) PTE LTD


In [7]:
dc = zip(date, description,amount)
df_data_transaction = pd.DataFrame(dc, columns=['Date','Desc','Amount'])
print("Name : ",name[0])
print("Address: ",address_suspect[0]) 
print("Account Number: ",suspect_account_number[0])
print("Statement Date: ",suspect_date2[0])
   
# print("", get_transaction)
df_data_transaction

Name :  JOHN RIEGER
Address:  KTGIF SINGAPORE PTE. LTD. 26B TEMPLE STREET #03-00 SINGAPORE 058571
Account Number:  12-145753-2
Statement Date:  31 Aug 2018


,Date,Desc,Amount
0,[28 Aug],"[28 Aug, Quick Cheque Deposit]",[]
1,[30 Aug],"[30 Aug, Point-of-Sale Transaction, TRANSIT LI...",[20.00]
2,[30 Aug],"[30 Aug, Point-of-Sale Transaction, S &amp; S ...",[465.00]
3,[31 Aug],"[31 Aug, Point-of-Sale Transaction, GAYATRI RE...",[26.50]
4,[31 Aug],"[31 Aug, Point-of-Sale Transaction, FOCUS NETW...",[16.00]
